# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 1 2023 12:31PM,256399,10,0086345840,ISDIN Corporation,Released
1,Feb 1 2023 12:31PM,256399,10,0086345843,ISDIN Corporation,Released
2,Feb 1 2023 12:31PM,256399,10,0086345839,ISDIN Corporation,Released
3,Feb 1 2023 12:31PM,256399,10,0086345841,ISDIN Corporation,Released
4,Feb 1 2023 12:31PM,256399,10,0086345842,ISDIN Corporation,Released
5,Feb 1 2023 12:31PM,256399,10,0086345850,ISDIN Corporation,Released
6,Feb 1 2023 12:31PM,256399,10,0086345856,ISDIN Corporation,Released
7,Feb 1 2023 12:31PM,256399,10,0086345859,ISDIN Corporation,Released
8,Feb 1 2023 12:31PM,256399,10,0086345861,ISDIN Corporation,Released
9,Feb 1 2023 12:31PM,256399,10,0086345860,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,256393,Released,5
23,256395,Released,5
24,256397,Released,1
25,256398,Released,1
26,256399,Released,22


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256393,NaN,8.0,5.0
256395,NaN,NaN,5.0
256397,NaN,NaN,1.0
256398,NaN,NaN,1.0
256399,NaN,NaN,22.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256319,19.0,3.0,0.0
256340,0.0,0.0,1.0
256350,0.0,0.0,1.0
256352,0.0,14.0,6.0
256353,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256319,19,3,0
256340,0,0,1
256350,0,0,1
256352,0,14,6
256353,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256319,19,3,0
1,256340,0,0,1
2,256350,0,0,1
3,256352,0,14,6
4,256353,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256319,19,3,
1,256340,,,1
2,256350,,,1
3,256352,,14,6
4,256353,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation
22,Feb 1 2023 12:21PM,256398,10,Emerginnova
23,Feb 1 2023 12:17PM,256397,19,"Granules Pharmaceuticals, Inc."
24,Feb 1 2023 12:15PM,256395,10,ISDIN Corporation
29,Feb 1 2023 11:26AM,256393,10,Eywa Pharma Inc.
42,Feb 1 2023 11:15AM,256392,21,"NBTY Global, Inc."
43,Feb 1 2023 11:12AM,256391,21,"NBTY Global, Inc."
44,Feb 1 2023 11:07AM,256390,21,"NBTY Global, Inc."
45,Feb 1 2023 10:06AM,256383,10,"ClearSpec, LLC"
47,Feb 1 2023 9:39AM,256378,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation,,,22
1,Feb 1 2023 12:21PM,256398,10,Emerginnova,,,1
2,Feb 1 2023 12:17PM,256397,19,"Granules Pharmaceuticals, Inc.",,,1
3,Feb 1 2023 12:15PM,256395,10,ISDIN Corporation,,,5
4,Feb 1 2023 11:26AM,256393,10,Eywa Pharma Inc.,,8,5
5,Feb 1 2023 11:15AM,256392,21,"NBTY Global, Inc.",,,1
6,Feb 1 2023 11:12AM,256391,21,"NBTY Global, Inc.",,,1
7,Feb 1 2023 11:07AM,256390,21,"NBTY Global, Inc.",,,1
8,Feb 1 2023 10:06AM,256383,10,"ClearSpec, LLC",,,2
9,Feb 1 2023 9:39AM,256378,19,"GCH Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation,22,,
1,Feb 1 2023 12:21PM,256398,10,Emerginnova,1,,
2,Feb 1 2023 12:17PM,256397,19,"Granules Pharmaceuticals, Inc.",1,,
3,Feb 1 2023 12:15PM,256395,10,ISDIN Corporation,5,,
4,Feb 1 2023 11:26AM,256393,10,Eywa Pharma Inc.,5,8,
5,Feb 1 2023 11:15AM,256392,21,"NBTY Global, Inc.",1,,
6,Feb 1 2023 11:12AM,256391,21,"NBTY Global, Inc.",1,,
7,Feb 1 2023 11:07AM,256390,21,"NBTY Global, Inc.",1,,
8,Feb 1 2023 10:06AM,256383,10,"ClearSpec, LLC",2,,
9,Feb 1 2023 9:39AM,256378,19,"GCH Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation,22,,
1,Feb 1 2023 12:21PM,256398,10,Emerginnova,1,,
2,Feb 1 2023 12:17PM,256397,19,"Granules Pharmaceuticals, Inc.",1,,
3,Feb 1 2023 12:15PM,256395,10,ISDIN Corporation,5,,
4,Feb 1 2023 11:26AM,256393,10,Eywa Pharma Inc.,5,8,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation,22.0,NaN,NaN
1,Feb 1 2023 12:21PM,256398,10,Emerginnova,1.0,NaN,NaN
2,Feb 1 2023 12:17PM,256397,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
3,Feb 1 2023 12:15PM,256395,10,ISDIN Corporation,5.0,NaN,NaN
4,Feb 1 2023 11:26AM,256393,10,Eywa Pharma Inc.,5.0,8.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation,22.0,0.0,0.0
1,Feb 1 2023 12:21PM,256398,10,Emerginnova,1.0,0.0,0.0
2,Feb 1 2023 12:17PM,256397,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
3,Feb 1 2023 12:15PM,256395,10,ISDIN Corporation,5.0,0.0,0.0
4,Feb 1 2023 11:26AM,256393,10,Eywa Pharma Inc.,5.0,8.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1794700,77.0,19.0,0.0
15,769082,34.0,11.0,25.0
16,256340,1.0,0.0,0.0
19,1538149,9.0,18.0,19.0
21,769173,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1794700,77.0,19.0,0.0
1,15,769082,34.0,11.0,25.0
2,16,256340,1.0,0.0,0.0
3,19,1538149,9.0,18.0,19.0
4,21,769173,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,77.0,19.0,0.0
1,15,34.0,11.0,25.0
2,16,1.0,0.0,0.0
3,19,9.0,18.0,19.0
4,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,77.0
1,15,Released,34.0
2,16,Released,1.0
3,19,Released,9.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,0.0,25.0,0.0,19.0,0.0
Executing,19.0,11.0,0.0,18.0,0.0
Released,77.0,34.0,1.0,9.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,0.0,25.0,0.0,19.0,0.0
1,Executing,19.0,11.0,0.0,18.0,0.0
2,Released,77.0,34.0,1.0,9.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,0.0,25.0,0.0,19.0,0.0
1,Executing,19.0,11.0,0.0,18.0,0.0
2,Released,77.0,34.0,1.0,9.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()